In [1]:
import random
from networkx import Graph
import numpy as np
import pandas as pd
import operator
import time

First we define our class node which has a name and neighbor nodes, a method to get the time with its respective neighbors and its representation

In [2]:
class CityNode:
    def __init__(self, name, neighbors):
        self.name = name
        #neighbors is a dictionary where the keys are the names of the neighbor nodes and the values ​​are dictionaries containing attributes of the edges, such as travel time.
        self.neighbors = neighbors 
    def travel_time(self, node):
        return self.neighbors[node.name]['time'] #travel time to neighbours
    def __repr__(self):
        return "(" + str(self.name) + ")"

Then we define our graph 

In [3]:
city_network = Graph()
city_network.add_edge("Rosario", "Instituto del petroleo", time=6)
city_network.add_edge("Instituto del petroleo", "La Raza", time=2)
city_network.add_edge("La Raza", "Guerrero", time=2)
city_network.add_edge("Guerrero", "Hidalgo", time=1)
city_network.add_edge("Hidalgo", "Tacuba", time=7)
city_network.add_edge("Tacuba", "Rosario", time=4)
city_network.add_edge("Deportivo 18 de marzo", "Instituto del petroleo", time=2)
city_network.add_edge("Deportivo 18 de marzo", "La Raza", time=2)
city_network.add_edge("Martin Carrera", "Deportivo 18 de marzo", time=2)
city_network.add_edge("Martin Carrera", "Consulado", time=3)
city_network.add_edge("Consulado", "La Raza", time=3)
city_network.add_edge("Consulado", "Oceania", time=3)
city_network.add_edge("Consulado", "Morelos", time=2)
city_network.add_edge("Garibaldi", "Guerrero", time=1)
city_network.add_edge("Garibaldi", "Morelos", time=3)
city_network.add_edge("Oceania", "San Lazaro", time=3)
city_network.add_edge("Morelos", "San Lazaro", time=1)
city_network.add_edge("Pantitlan", "Oceania", time=3)
city_network.add_edge("Pantitlan", "San Lazaro", time=6)
city_network.add_edge("Candelaria", "San Lazaro", time=1)
city_network.add_edge("Candelaria", "Morelos", time=1)
city_network.add_edge("Pino Suarez", "Candelaria", time=2)
city_network.add_edge("Pino Suarez", "Bellas artes", time=3)
city_network.add_edge("Bellas artes", "Garibaldi", time=1)
city_network.add_edge("Bellas artes", "Hidalgo", time=1)
city_network.add_edge("Salto del agua", "Bellas artes", time=2)
city_network.add_edge("Salto del agua", "Pino Suarez", time=2)
city_network.add_edge("Balderas", "Hidalgo", time=2)
city_network.add_edge("Balderas", "Salto del agua", time=1)
city_network.add_edge("Balderas", "Tacubaya", time=6)
city_network.add_edge("Tacuba", "Tacubaya", time=5)
city_network.add_edge("Centro medico", "Tacubaya", time=3)
city_network.add_edge("Centro medico", "Balderas", time=3)
city_network.add_edge("Centro medico", "Chabacano", time=2)
city_network.add_edge("Salto del agua", "Chabacano", time=3)
city_network.add_edge("Pino Suarez", "Chabacano", time=2)
city_network.add_edge("Jamaica", "Candelaria", time=2)
city_network.add_edge("Jamaica", "Chabacano", time=1)
city_network.add_edge("Jamaica", "Pantitlan", time=5)
city_network.add_edge("Baja jamaica", "Chabacano", time=2)
city_network.add_edge("Baja jamaica", "Jamaica", time=1)
city_network.add_edge("Tacubaya", "Mixcoac", time=3)
city_network.add_edge("Zapata", "Mixcoac", time=3)
city_network.add_edge("Zapata", "Centro medico", time=4)
city_network.add_edge("Zapata", "Ermita", time=3)
city_network.add_edge("Ermita", "Chabacano", time=6)
city_network.add_edge("Ermita", "Atlalilco", time=2)
city_network.add_edge("Baja jamaica", "Atlalilco", time=6)

Now we can see the list of nodes that we have

In [4]:
#Creates a list of nodes iterating over the city_network nodes and adding its neighbours iterating over the neighbors of each node.
city_list = [CityNode(node, {neighbor: city_network.edges[node, neighbor] for neighbor in city_network.neighbors(node)}) for node in city_network.nodes()]

print(str(city_list))

[(Rosario), (Instituto del petroleo), (La Raza), (Guerrero), (Hidalgo), (Tacuba), (Deportivo 18 de marzo), (Martin Carrera), (Consulado), (Oceania), (Morelos), (Garibaldi), (San Lazaro), (Pantitlan), (Candelaria), (Pino Suarez), (Bellas artes), (Salto del agua), (Balderas), (Tacubaya), (Centro medico), (Chabacano), (Jamaica), (Baja jamaica), (Mixcoac), (Zapata), (Ermita), (Atlalilco)]


Lets define the the departure node and the destination node, based on the list of the nodes list

In [5]:
def find_index(node: str) -> int:
    for i in range(len(city_list)):
        if(node in city_list[i].name): #if the name node nodes is in the list of nodes return index
            return i
        
start_node_index = find_index("Rosario") #start node of the problem
end_node_index = find_index("San Lazaro") #end node of the problem

Then we have our initialPopulation and generateGenome function, the function initialpopulation will create genomes based on the population size, is just a for loop that calls the generate genome function, which generates random routes from the departure node to the destination node.

In [6]:
def generate_genome_sequence(city_list):
    genome = [] #Initialize 
    #Create a copy of the nodelist to manipulate the list without causing changes in the original
    remaining_nodes = city_list.copy()
    #initialize the start of the genome with the departure node
    current_node = remaining_nodes[start_node_index]
    #remove node of the remaining nodes
    remaining_nodes.remove(current_node)
    #append the current node that is the start of the route to the genome
    genome.append(current_node)

    while remaining_nodes:
      #the avaible nodes are the nodes that are neighbors to the current node
      available_nodes = [node for node in remaining_nodes if node.name in current_node.neighbors]

      if not available_nodes:# if no available cities, return None
          return None 
      
      #chose a random neighbor node
      next_node = random.choice(available_nodes) 
      #if the nex_node is the destination node, return the genome as a possible solution
      if(next_node == city_list[end_node_index]):
          genome.append(next_node)
          return genome
      else:#else append the next_node or gene to genome and continue generating genome
        genome.append(next_node)
        remaining_nodes.remove(next_node)
        current_node = next_node

    return genome

def initialize_population(popSize, city_list):
    #initialize the list of genomes known as our population
    population = []
    #while population is not complete
    while len(population) < popSize:
        #create a genome
        genome = generate_genome_sequence(city_list)
        #if genome is not none append the genome to the population
        if genome is not None:
            population.append(genome)
    return population

Our fitness class that will check the fitness of the genome, we sum the times of the genome, and then return the inverse of time since the shorter the route time, the greater the fitness.

In [7]:
class GenomeFitness:
    def __init__(self, genome):
        self.genome = genome
        #initialize time in 0
        self.time = 0
        #initialize fitness in 0
        self.fitness = 0.0

    def routeDistance(self):
        #initialize the time of the genome as 0
        path_time = 0
        #we Iterate across the genome 
        for i in range(0, len(self.genome)):
            #set a variable for the node that we are evaluating in the iteration
            fromCityNode = self.genome[i]
            #if there is a next node in the genome
            if i + 1 < len(self.genome):
                #set a variable for the next node
                toCity = self.genome[i + 1]
            #we ensure that the next node is in the neighbors of the nodes that we are evaluating
            if toCity.name in fromCityNode.neighbors:
                #add the travel time to the path time
                path_time += fromCityNode.travel_time(toCity)
        self.time = path_time
        #return the path time
        return  self.time

    def routeGenomeFitness(self):
        if self.fitness == 0:
            #set a variable for the path time
            time = self.routeDistance()
            #set the fitness as the inverse of the time
            self.fitness = 1 / float(time)
        return self.fitness

The rankGenomes function will help us to rank the genomes by their fitness function, meaning that the first genomes are closer to the solution compared to the others.

In [8]:
def rank_genome_by_fitness(population):
    #initialize the list of the fitness results
    fitnessResults = {}
    for i in range(0,len(population)):
        #add the result of the currente genome in the population
        fitnessResults[i] = GenomeFitness(population[i]).routeGenomeFitness()
    #sort the genomes based on their fitness
    sorted_results=sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)
    return sorted_results

The selection function will preserve the elite that are the best genomes, and will select what others genomes are preserved

In [9]:
def select_elite_genomes(popRanked, eliteSize):
    #initialize list of select_elite_genomes results
    select_elite_genomesResults = []
    #create a dataframe of the population ranked with columns of its index and fitness
    df = pd.DataFrame(np.array(popRanked), columns=["Index","GenomeFitness"])
    #calculate the fitness accumulated sum 
    df['cum_sum'] = df.GenomeFitness.cumsum()
    #percentage of cumulative fitness value
    df['cum_perc'] = 100*df.cum_sum/df.GenomeFitness.sum()
    #preserve the elite of the population
    for i in range(0, eliteSize):
        select_elite_genomesResults.append(popRanked[i][0])
    #iterate over the rest
    for i in range(0, len(popRanked) - eliteSize):
        #chose a random genome of the population
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            #compares the pick and the accumulated percentage sum, if the pick i lesser, we chose that genome
            if pick <= df.iat[i,3]:
                select_elite_genomesResults.append(popRanked[i][0])
                break
    return select_elite_genomesResults

The matingpool function creates a mating pool by selecting individuals from the population according to previously calculated selection results.

In [10]:
def create_mating_pool(population, select_elite_genomesResults):
    #Initialize list of the mating pool
    matingpool = []
    #iterate over the select_elite_genomes results
    for i in range(0, len(select_elite_genomesResults)):
        index = select_elite_genomesResults[i]
        #append each selected genomes to the mating pool
        matingpool.append(population[index])
    return matingpool

In the function crossover makes the child of two genomes selected

In [11]:
def combine_genomes(parent1, parent2):
    #Initialize the genome of the offspring from two parents
    child = []
    #first half of child
    childP1 = []
    #second half of child
    childP2 = []
    #select two random points for the cross overs
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    #determine the points of start and end for the combine_genomes
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    #create the first half of the genome
    for i in range(startGene, endGene):
        childP1.append(parent1[i])
    #create the second half of the child   
    childP2 = [item for item in parent2 if item not in childP1]
    #join the two parts
    child = childP1 + childP2
    return child

the cross_population function cross the genomes in the matingpool list, and returns a list of the childrens

In [12]:
def crossover_population(matingpool, eliteSize):
    #initialize list of children
    children = []
    #the length of children is the mating pool minus the size of the elite
    length = len(matingpool) - eliteSize
    # The mating group is randomly mixed
    pool = random.sample(matingpool, len(matingpool))
    #append the elite in the children list since they are gonna pass to the next generation
    for i in range(0,eliteSize):
        children.append(matingpool[i])
    #generate children of no elite genomes
    for i in range(0, length):
        #set a variable for the child of two parents at different sides of the matingpool
        child = combine_genomes(pool[i], pool[len(matingpool)-i-1])
        #verify that all the nodes in the genome are neighbors
        valid_child = all(node1.name in node2.neighbors for node1, node2 in zip(child[:-1], child[1:]))
        #if is a valid child append to the childrens list
        if valid_child:
            children.append(child)
        else:
            #try to generate other children with other parents
            continue
    return children

The mutate function, takes the mutation rate, and if a random number is lesser than the mutation rate, does the mutation, a mutation is to change genes of the genome.

In [13]:
def apply_mutation(individual, mutationRate):
    #Iterate across the genome
    for swapped in range(len(individual)):
        #if a random number is less than the mutation rate
        if(random.random() < mutationRate):
            #the mutation, the gene that change
            swapWith = int(random.random() * len(individual))
            #set variables for the genes tobe swapped
            node1 = individual[swapped]
            node2 = individual[swapWith]
            # Verify that the adjacent nodes are neighbors of the apply_mutationd gene
            if (swapped == 0 or node2.name in individual[swapped - 1].neighbors) and \
               (swapped == len(individual) - 1 or node2.name in individual[swapped + 1].neighbors) and \
               (swapWith == 0 or node1.name in individual[swapWith - 1].neighbors) and \
               (swapWith == len(individual) - 1 or node1.name in individual[swapWith + 1].neighbors):
            #do the exchange of genes
                individual[swapped] = node2
                individual[swapWith] = node1
            else:
              #continue with other gene without doing changes since the mutation was not valid
              continue
    return individual

the mutate population function just calls the mutate function for all the population

In [14]:
def apply_mutationPopulation(population, mutationRate):
    #Initialize list of the apply_mutationd population
    apply_mutationdPop = []
    #iterate across the population
    for ind in range(0, len(population)):
        #call the apply_mutation function and set a variable to keep the return of the function
        #the apply_mutationd genome
        apply_mutationdInd = apply_mutation(population[ind], mutationRate)
        #append the apply_mutationd genome to the list
        apply_mutationdPop.append(apply_mutationdInd)
    return apply_mutationdPop

the next generation function returns the population of the next generation calling the methods rankGenomes, selection, matingPool, cross_Population, mutatePopulation.

In [15]:
def create_next_generation(currentGen, eliteSize, mutationRate):
    #rank generation
    popRanked = rank_genome_by_fitness(currentGen)
    #select the genomes of the current generation
    select_elite_genomesResults = select_elite_genomes(popRanked, eliteSize)
    #set the mating pool
    matingpool = create_mating_pool(currentGen, select_elite_genomesResults)
    #cross parents in mating pool
    children = crossover_population(matingpool, eliteSize)
    #apply_mutation the children
    create_next_generation = apply_mutationPopulation(children, mutationRate)
    return create_next_generation

The function of the genetic algorithm, receives the nodeList, the size of the population, the size of the elite, the mutation rate and the generations, will run the nextGeneration method the generation that we specify and save the best genome in each generation.

In [16]:
def run_genetic_algorithm(city_list, popSize, eliteSize, mutationRate, generations):
    #start a initial population
    pop = initialize_population(popSize, city_list)
    #list to store the best fitness in each generation
    progress = []
    #rank the best fitness in the first population and append it to the progress list
    progress.append(rank_genome_by_fitness(pop)[0][1])
    #run x number of generations
    for i in range(0, generations):
        #the population of the next generation
        pop = create_next_generation(pop, eliteSize, mutationRate)
        #the best fitness of current generation
        best_fitness = rank_genome_by_fitness(pop)[0][1]
        #append the best fitness to the progress list
        progress.append(best_fitness)
        #get the index of the best genome
        best_genome_index = rank_genome_by_fitness(pop)[0][0]
        #get the best genome
        best_genome = pop[best_genome_index]
        #get the fitness of the best genome
        best_genome_fitness = GenomeFitness(best_genome)
        #get time of best genome
        best_genome_time = best_genome_fitness.routeDistance()

    #get the best genome index after all the generations
    best_genome_index = rank_genome_by_fitness(pop)[0][0]
    #get the best genome  after all the generations
    best_genome = pop[best_genome_index]
    return best_genome

debug the execution time and launch the algorithm with given parameters

In [17]:
start = time.time()
best_genome = run_genetic_algorithm(city_list=city_list, popSize=100, eliteSize=40, mutationRate=0.3, generations=50)

#get the time of the best genome
best_genome_fitness = GenomeFitness(best_genome)
best_genome_time = best_genome_fitness.routeDistance()

print("")
print("Best Route:")
print([city.name for city in best_genome])
print("Best Route Time:", best_genome_time)
end = time.time()
print("Execution time: "+ str(end-start))



Best Route:
['Rosario', 'Instituto del petroleo', 'La Raza', 'Consulado', 'Morelos', 'San Lazaro']
Best Route Time: 14
Execution time: 0.08253002166748047
